## Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))

import ripso
import optimizationFuncs as optim

## Reload (if necessary)

In [ ]:
from importlib import reload
ripso = reload(ripso)
optim = reload(optim)

## Setup

In [ ]:
func = optim.schaffer4
funckey = lambda x : func(x.reshape(1, -1))[0]
Np = 25
D = 2
llim = np.repeat(-10, D)
rlim = np.repeat(10, D)
mizer = ripso.RI_PSO(func, llim, rlim, Np)

## optimize()

In [ ]:
opt = mizer.optimize()
print(opt)

## Plot Optima and Hulls

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_figheight(7.5)
fig.set_figwidth(7.5)

for hull in mizer.hulls :
    hull, pip, hb = (hull.hull, hull.pip, hull.verts)
    ax.plot(pip[0], pip[1], 'ro', label='global optima')
    ax.plot(hb[:,0], hb[:,1], 'ko', label='reverse-PSO final position')
    for i,j in zip(hull.vertices[:-1], hull.vertices[1:]) :
        x = [hb[i,0], hb[j,0]]
        y = [hb[i,1], hb[j,1]]
        ax.plot(x, y, 'b')

    i = hull.vertices[-1]
    j = hull.vertices[0]
    x = [hb[i,0], hb[j,0]]
    y = [hb[i,1], hb[j,1]]
    ax.plot(x, y, 'b')